## Imports

In [1]:
import pandas as pd
import sys, os, importlib
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', 100)

if os.path.join('..','0_funcoes_base') not in sys.path:
    sys.path.append(os.path.join('..','0_funcoes_base')) 

df_manipulator = importlib.import_module('df_manipulator')
date_manipulator = importlib.import_module('date_manipulator')
file_manipulator = importlib.import_module('file_manipulator')

## Dados de configuração

In [3]:
input_config = {
    'file': {
        'ref_dir':'../../1_dados_raw/',
        'filename':'export_lunar_20200517.csv',
        'delimiter': ';',
        'header': ['id', 'speedBumpId','timestamp','x','y','z','lat','lng']
    }
}

output_config = {
    'file': {
        'ref_dir': './out',
        'prefix': '1_',
        'delimiter':';',
        'with_header': True
    }
}

## Funções

In [21]:
def remove_invalid_timestamps(df):
    df = df_manipulator.filter_by_dates(df, start_date='2000-01-01', end_date=datetime.utcnow(), format='%Y-%m-%d')
    return df

def update_timestamp_ticks_to_datetime(df):
    df['new_timestamp'] = df.apply(lambda row: date_manipulator.ticks_to_datetime(row.timestamp), axis=1)

    df = df_manipulator.remove_column(df, 'timestamp', False)
    df.rename(columns={'new_timestamp':'timestamp'},inplace=True)

    df_manipulator.set_index(df, 'timestamp', True)
    df.sort_index(inplace=True)

    return df

def remove_invalid_accelerometer_values(df, filter_columns, invalid_value = 0):
    _df = df.copy()
    
    if len(filter_columns) > 0:
        for filter_column in filter_columns:
            _df = _df[_df[filter_column] != invalid_value]
    return _df

def remove_duplicated_timestamp(df):
    _df = df.copy()

    # Set 'timestamp' column as index
    if _df.index.name != 'timestamp':
        _df.set_index('timestamp', inplace=True)
    
    # Put duplicated timestamp with speedBump event to last occurrence
    _df.sort_values(by=['timestamp', 'speedBumpId'], inplace=True)

    # Remove duplicated timestamp
    _df = _df.loc[~_df.index.duplicated(keep='last')]

    return _df

## Carrega dataframe bruto

In [5]:
df = df_manipulator.load_dataframe(input_config['file']['filename'], input_config['file']['ref_dir'], input_config['file']['delimiter'], input_config['file']['header'])

## Dataframe bruto

In [6]:
df.speedBumpId.value_counts()

0    11084
1       42
2       10
Name: speedBumpId, dtype: int64

In [7]:
df

,id,speedBumpId,timestamp,x,y,z,lat,lng
0,d198e52b-bd5b-424a-8ea5-d391fbdb7b20,0,1589757048346,-3.399200,9.516205,1.573975,-22.921991,-42.473372
1,a9df8305-e4ed-4530-ab78-e1aa3ca9ffbd,0,1589757048543,-2.987396,10.203339,1.291458,-22.921991,-42.473372
2,74dedc33-8d9b-4aa6-a352-df1ee9364ed7,0,1589757048743,-4.376038,9.746048,1.806213,-22.921991,-42.473372
3,4a64f1bf-ea98-49db-baff-d7d08baf2dce,0,1589757048843,-3.698471,9.317490,1.849304,-22.921991,-42.473372
4,25a3976b-a812-4bf2-a2fe-113d7b01cd11,0,1589757048944,-5.039230,9.372559,0.522919,-22.921991,-42.473372
...,...,...,...,...,...,...,...,...
11131,0f15d4aa-c4c3-40ba-9d1f-1746e938a5eb,0,1589758551161,0.386032,10.105179,1.660172,-22.925816,-42.484631
11132,c044a484-bb8b-421b-8cc8-a6ad4713e28f,0,1589758551261,-1.378494,9.920822,2.608276,-22.925816,-42.484631
11133,c9d94f9a-6211-4198-b39b-70db0ef2e890,0,1589758551361,-1.215683,9.523392,2.505325,-22.925816,-42.484631
11134,2fa87037-71b8-42e0-bd89-64bff33cfead,0,1589758551461,-0.435181,8.999054,2.871628,-22.925816,-42.484631


## Detalhes do dataframe bruto

In [8]:
df_manipulator.apply_describe(df)

Shape:
 (11136, 8)
Types:

float64 :  ['x' 'y' 'z' 'lat' 'lng']
int64 :  ['speedBumpId' 'timestamp']
object :  ['id']





,null_sum,null_pct,dtypes,count,mean,median,min,max
id,0,0.0,object,11136,NaN,NaN,0000103f-58f2-4b67-956e-9fc49ea2a845,ffe9ed07-2030-4a2d-ad61-18af6217f012
lat,0,0.0,float64,11136,-2.291595e+01,-2.291711e+01,-22.9355,-22.8901
lng,0,0.0,float64,11136,-4.247970e+01,-4.247697e+01,-42.4957,-42.4683
speedBumpId,0,0.0,int64,11136,5.567529e-03,0.000000e+00,0,2
timestamp,0,0.0,int64,11136,1.589758e+12,1.589758e+12,1589757048346,1589758551561
x,0,0.0,float64,11136,-6.641728e-01,-6.338959e-01,-12.7126,5.52878
y,0,0.0,float64,11136,9.331052e+00,9.379730e+00,3.04707,15.6933
z,0,0.0,float64,11136,2.894415e+00,2.696854e+00,-8.12733,15.8889


## Remove NaN

In [9]:
df_out = df_manipulator.remove_nan(df)

## Remove timestamp inválido e muda formato

In [10]:
df_out = update_timestamp_ticks_to_datetime(df_out)
df_out = remove_invalid_timestamps(df_out)

## Remove valores de acelerômetro inválidos

In [11]:
df_out = remove_invalid_accelerometer_values(df_out, filter_columns=['x','y','z'], invalid_value=0)

## Remove timestamp duplicado

In [12]:
df_out = remove_duplicated_timestamp(df_out)

## Dataframe filtrado

In [13]:
df_out

,id,speedBumpId,x,y,z,lat,lng
timestamp,,,,,,,
2020-05-17 20:10:48.346,d198e52b-bd5b-424a-8ea5-d391fbdb7b20,0,-3.399200,9.516205,1.573975,-22.921991,-42.473372
2020-05-17 20:10:48.543,a9df8305-e4ed-4530-ab78-e1aa3ca9ffbd,0,-2.987396,10.203339,1.291458,-22.921991,-42.473372
2020-05-17 20:10:48.743,74dedc33-8d9b-4aa6-a352-df1ee9364ed7,0,-4.376038,9.746048,1.806213,-22.921991,-42.473372
2020-05-17 20:10:48.843,4a64f1bf-ea98-49db-baff-d7d08baf2dce,0,-3.698471,9.317490,1.849304,-22.921991,-42.473372
2020-05-17 20:10:48.944,25a3976b-a812-4bf2-a2fe-113d7b01cd11,0,-5.039230,9.372559,0.522919,-22.921991,-42.473372
...,...,...,...,...,...,...,...
2020-05-17 20:35:51.161,0f15d4aa-c4c3-40ba-9d1f-1746e938a5eb,0,0.386032,10.105179,1.660172,-22.925816,-42.484631
2020-05-17 20:35:51.261,c044a484-bb8b-421b-8cc8-a6ad4713e28f,0,-1.378494,9.920822,2.608276,-22.925816,-42.484631
2020-05-17 20:35:51.361,c9d94f9a-6211-4198-b39b-70db0ef2e890,0,-1.215683,9.523392,2.505325,-22.925816,-42.484631


## Detalhes do dataframe filtrado

In [14]:
df_out.speedBumpId.value_counts()

0    11032
1       42
2       10
Name: speedBumpId, dtype: int64

In [17]:
df_manipulator.apply_describe(df_out)

Shape:
 (11084, 7)
Types:

float64 :  ['x' 'y' 'z' 'lat' 'lng']
int64 :  ['speedBumpId']
object :  ['id']





,null_sum,null_pct,dtypes,count,mean,median,min,max
id,0,0.0,object,11084,NaN,NaN,0000103f-58f2-4b67-956e-9fc49ea2a845,ffe9ed07-2030-4a2d-ad61-18af6217f012
lat,0,0.0,float64,11084,-22.915948,-22.917111,-22.9355,-22.8901
lng,0,0.0,float64,11084,-42.479699,-42.476972,-42.4957,-42.4683
speedBumpId,0,0.0,int64,11084,0.005594,0.000000,0,2
x,0,0.0,float64,11084,-0.664460,-0.635094,-12.7126,5.52878
y,0,0.0,float64,11084,9.333137,9.382126,3.04707,15.6933
z,0,0.0,float64,11084,2.891374,2.695656,-8.12733,15.8889


## Salva o dataframe de saída

In [20]:
out_filename = file_manipulator.get_out_filename(output_config['file']['prefix'], output_config['file']['ref_dir'], input_config['file']['filename'])

df_out.to_csv(out_filename, sep=output_config['file']['delimiter'], header=output_config['file']['with_header'])

print(f'O arquivo {out_filename} foi gerado!')

O arquivo ./out/1_export_lunar_20200517.csv foi gerado!
